https://www.tsbpa.texas.gov/php/fpl/indlookup.php





5:51
Scrape only for Anderson county.
5:53
This is supposed to be tricky. My suggestion is to use requests and beautifulsoup for debugging. Then make a scrapy spider of it.
5:56
- Certificate Last Name
- City
- State
- Date Certified
- Date License Expiration
- Status
- Firms in which the individual is a partner, shareholder, owner, officer, director, or resident manager (This is just one string, no need to extract further details from it)

In [1]:
import requests
from bs4 import BeautifulSoup

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36 Edg/137.0.0.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cookie': 'ZDEDebuggerPresent=php,phtml,php3; PHPSESSID=e81Snkuip6euhem30723qquh21h2gp6f',
    'Host': 'www.tsbpa.texas.gov',
    'Origin': 'https://www.tsbpa.texas.gov',
    'Referer': 'https://www.tsbpa.texas.gov/php/fpl/indlookup.php',
    'Sec-Ch-Ua': '"Microsoft Edge";v="137", "Chromium";v="137", "Not(A:Brand";v="24"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1'
}

session = requests.Session()
session.headers.update(headers)

In [9]:
response  = session.get('https://www.tsbpa.texas.gov/php/fpl/indlookup.php',timeout=10)
response.status_code

200

In [10]:
soup = BeautifulSoup(response.text, 'html.parser')

In [11]:
import pprint
form = soup.find('form')

form_data = {}
for input_tag in form.find_all(['input','select']):
    if input_tag.get('name'):
        form_data[input_tag.get('name')] = input_tag.get('value', '')
form_data['CNTY']='ANDERSON'

form_data['submit'] = 'Submit Search'

# Make sure we have empty values for other fields
form_data['LICID'] = ''
form_data['LNAME'] = ''
form_data['FNAME'] = ''
form_data['CLNAME'] = ''
form_data['CITY'] = ''
form_data['STATE'] = ''
form_data['ZIP'] = ''
pprint.pprint(form_data)

{'CITY': '',
 'CLNAME': '',
 'CNTY': 'ANDERSON',
 'FNAME': '',
 'LICID': '',
 'LNAME': '',
 'STATE': '',
 'ZIP': '',
 'list': 'fromsel',
 'submit': 'Submit Search',
 'tsbpa684f0c364d1df': 'tsbpa684f0c364d1df'}


In [12]:
submit_response = session.post(
    'https://www.tsbpa.texas.gov/php/fpl/indlookup.php',
    data=form_data,timeout=10
)
print(submit_response.status_code)


200


In [13]:
submit_soup = BeautifulSoup(submit_response.text, 'html.parser')


In [14]:
table = submit_soup.find_all("table")[5]

In [62]:
import json


count = 0
with open('data.jsonl', 'w') as f:
    for i in table.find_all('tr'):
        req_href = i.find('td').find('a')['href']
        url = f'https://www.tsbpa.texas.gov/php/fpl/{req_href}'
        response2= session.get(url)
        response2_soup = BeautifulSoup(response2.text, 'html.parser')
        table2 = response2_soup.find_all('table')[3]
        
        certificate_last_name = table2.find_all('tr')[4].text
        city = table2.find_all('tr')[6].text.split(',')[0]
        state = table2.find_all('tr')[6].text.split(',')[1]
        registered_date =table2.find_all('tr')[10].text 
        expiration_date = table2.find_all('tr')[13].text
        status = table2.find_all('tr')[16].text
        firms = table2.find_all('tr')[26].text
        
        data = {
            'count': count,
            'certificate_last_name': certificate_last_name.strip(),
            'city': city.strip(),
            'state': state.strip(),
            'registered_date': registered_date.strip(),
            'expiration_date': expiration_date.strip(),
            'status': status.strip(),
            'firms': firms.strip()
        }
            
        f.write(json.dumps(data) + '\n')
        count += 1
    
    